In [1]:
# Basic kernel test
import sys
print(f"Python version: {sys.version}")
print(f"Python executable: {sys.executable}")
print("Kernel is working!")

Python version: 3.14.0 | packaged by Anaconda, Inc. | (main, Oct  8 2025, 17:15:28) [GCC 11.2.0]
Python executable: /home/guru/miniconda3/envs/DINO/bin/python
Kernel is working!


## Quick Test - Kernel Check

Run this first to verify the kernel works:

# Dinosaur Game Environment with Pygame
## Part 1: Setup and Game Implementation

This notebook implements the Chrome Dinosaur game environment using Pygame, compatible with OpenAI Gym interface for reinforcement learning.

## 1. Install and Import Required Libraries

In [2]:
# Import the game module and other required libraries
import sys
import os
sys.path.insert(0, '/mnt/c/Dino RL')

# Prevent pygame from crashing in WSL/notebook environment
# Set to use dummy video driver if no DISPLAY (will be overridden if render=True)
if not os.environ.get('DISPLAY'):
    os.environ['SDL_VIDEODRIVER'] = 'dummy'

from src.dino_game import Dinosaur, Obstacle, DinoGame
import pygame
import numpy as np
import gymnasium as gym
from gymnasium import spaces

# Display versions
print(f"Pygame version: {pygame.version.ver}")
print(f"NumPy version: {np.__version__}")
print(f"Gymnasium version: {gym.__version__}")
print("Game module imported successfully!")

pygame 2.6.1 (SDL 2.0.20, Python 3.14.0)
Hello from the pygame community. https://www.pygame.org/contribute.html
Pygame version: 2.6.1
NumPy version: 2.3.4
Gymnasium version: 1.2.1
Game module imported successfully!


<frozen importlib._bootstrap>:491: RuntimeWarning: Your system is avx2 capable but pygame was not built with support for it. The performance of some of your blits could be adversely affected. Consider enabling compile time detection with environment variables like PYGAME_DETECT_AVX2=1 if you are compiling without cross compilation.
/home/guru/miniconda3/envs/DINO/lib/python3.14/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


## 2. Game Classes (Imported from Module)

The game implementation (Dinosaur, Obstacle, DinoGame) is now in `src/dino_game.py` and imported above.

## 3. Create Gym Environment Wrapper

In [3]:
class DinoEnv(gym.Env):
    """
    OpenAI Gym compatible environment for Dino game
    """
    metadata = {'render.modes': ['human', 'none']}
    
    def __init__(self, render_mode='human'):
        super(DinoEnv, self).__init__()
        
        # Define action and observation space
        # Actions: 0 = do nothing, 1 = jump, 2 = duck
        self.action_space = spaces.Discrete(3)
        
        # Observation space: 8 features
        self.observation_space = spaces.Box(
            low=np.array([0, -1, 0, 0, 0, 0, 0, 0], dtype=np.float32),
            high=np.array([1, 1, 1, 1, 1, 1, 1, 1], dtype=np.float32),
            dtype=np.float32
        )
        
        # DinoGame uses 'render' parameter (True/False), not 'render_mode'
        should_render = (render_mode == 'human')
        self.game = DinoGame(render=should_render)
        
    def reset(self):
        """Reset the environment"""
        return self.game.reset()
        
    def step(self, action):
        """Execute action in environment"""
        return self.game.step(action)
        
    def render(self, mode='human'):
        """Render the environment"""
        self.game.render()
        
    def close(self):
        """Clean up resources"""
        self.game.close()

print("DinoEnv Gym wrapper created!")

DinoEnv Gym wrapper created!


## 4. Test the Environment - Random Agent

In [ ]:
# Test the environment with a random agent
print("Testing environment with random agent...")
print("This will run for a few episodes. Close the window or wait for game over.")

env = DinoEnv(render_mode='human')

# Run a few episodes
num_episodes = 3
for episode in range(num_episodes):
    obs = env.reset()
    done = False
    total_reward = 0
    steps = 0
    
    print(f"\n=== Episode {episode + 1} ===")
    
    while not done and steps < 1000:  # Limit steps per episode
        # Handle pygame events
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                done = True
                break
                
        # Take random action
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        total_reward += reward
        steps += 1
        
        # Render
        env.render()
        
        if done:
            print(f"Episode finished!")
            print(f"Final Score: {info['score']}")
            print(f"Total Reward: {total_reward:.2f}")
            print(f"Steps: {steps}")
            break
            
env.close()
print("\nEnvironment test complete!")

: 

## 5. Test Manual Control (Optional)

You can manually control the dinosaur with keyboard:

In [ ]:
# Manual control test - play the game yourself!
print("Manual Control Test")
print("Controls: SPACE or UP arrow to jump, DOWN arrow to duck")
print("Close the window to stop")

env = DinoEnv(render_mode='human')
obs = env.reset()
done = False
total_reward = 0

running = True
while running and not done:
    action = 0  # Default: do nothing
    
    # Handle events
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            
    # Check keyboard state
    keys = pygame.key.get_pressed()
    if keys[pygame.K_SPACE] or keys[pygame.K_UP]:
        action = 1  # Jump
    elif keys[pygame.K_DOWN]:
        action = 2  # Duck
        
    # Step environment
    obs, reward, done, info = env.step(action)
    total_reward += reward
    
    # Render
    env.render()
    
    if done:
        print(f"\nGame Over!")
        print(f"Final Score: {info['score']}")
        print(f"Total Reward: {total_reward:.2f}")
        print(f"Frames Survived: {info['frames']}")

env.close()
print("Manual control test complete!")

## Summary

✅ **Environment Complete!**

We've created:
1. **Dinosaur class** - Character with jump and duck actions
2. **Obstacle class** - Cacti and birds that move towards the dino
3. **DinoGame class** - Main game logic with physics and collision detection
4. **DinoEnv** - OpenAI Gym wrapper for RL training
5. **State space** - 8 features describing game state
6. **Action space** - 3 actions (nothing, jump, duck)
7. **Reward system** - +0.1 per frame, +10 per obstacle passed, -100 for collision

**Next Steps:**
- Create the DQN model notebook
- Train the agent using this environment
- Optimize hyperparameters for better performance